In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [2]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Song")])

AIMessage(content='Nice to meet you, Song! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 12, 'total_tokens': 26}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8a0de92f-5078-4e59-bb49-6e97f08fece4-0', usage_metadata={'input_tokens': 12, 'output_tokens': 14, 'total_tokens': 26})

The model on its own does not have any concept of state.

In [3]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not have access to personal information. Could you please tell me your name?", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 12, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0c0ed544-c237-46e6-9de3-063a8bcc24d3-0', usage_metadata={'input_tokens': 12, 'output_tokens': 21, 'total_tokens': 33})

In [12]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm reverg"),
        AIMessage(content="Hello reverg! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is reverg. How can I help you today, reverg?', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 37, 'total_tokens': 53}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-755da34c-cd04-4501-aa62-b35486913331-0', usage_metadata={'input_tokens': 37, 'output_tokens': 16, 'total_tokens': 53})

In [3]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [9]:
config = {"configurable": {"session_id": "reverg"}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I am reverg")],
    config=config,
)

response.content

'Hello, Reverg! How can I assist you today?'

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Reverg.'

In [13]:
config = {"configurable": {"session_id": "qwerty"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I can't know your name as I am an AI assistant and do not have access to personal information."

In [14]:
config = {"configurable": {"session_id": "reverg"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Reverg.'

In [16]:
print(store)

{'reverg': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am reverg'), AIMessage(content='Hello, Reverg! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 13, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4bf98100-b5c0-492d-89a3-0a37cfb05818-0', usage_metadata={'input_tokens': 13, 'output_tokens': 12, 'total_tokens': 25}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Reverg.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 38, 'total_tokens': 44}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0efd3970-4243-494c-a69c-cdd1cfe1d929-0', usage_metadata={'input_tokens': 38, 'output_tokens': 6, 'total_tokens': 44}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Reverg.', respons

In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [35]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm dvorak")]})

response.content

'Hello! How can I assist you today, Dvorak?'

In [62]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [37]:
config = {"configurable": {"session_id": "dv"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Dvorak")],
    config=config,
)

response.content

'Hello, Dvorak! How can I assist you today?'

In [38]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Dvorak.'

In [70]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [59]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm reverg")], "language": "Korean"}
)

response.content

'안녕하세요! reverg님, 반가워요. 어떻게 도와드릴까요?'

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [9]:
config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Korean"},
    config=config,
)

response.content

'안녕하세요, Todd님! 어떻게 도와드릴까요?'

In [10]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Korean"},
    config=config,
)

response.content

'당신의 이름은 Todd입니다.'

In [13]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [14]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't have access to personal information. How can I assist you today?"

In [15]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked for the result of 2 + 2.'

In [16]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [17]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't have access to that information. How can I assist you today?"

In [18]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"You haven't asked a specific math problem yet. Feel free to ask any math-related question you have, and I'll be happy to help!"

In [19]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

|Hi| Todd|!| Sure|,| here|'s| a| joke| for| you|:| Why| couldn|'t| the| bicycle| stand| up| by| itself|?| Because| it| was| two| tired|!| 😄||